$$\newcommand{\isum}{\sum_{i}}$$
$$\newcommand{\zsum}{\sum_{k=1}^{K}}$$
$$\newcommand{\zsumi}{\sum_{\{z_i\}}}$$

## Why does EM work?

To understand why EM works we will start with our old friend the KL-divergence. All images in this section are stolen from Bishop

We cast the problem as an MLE problem, but in general any problem in which there is a $\theta, x, z$ triad will work.

#### Word
Assume the hidden data $Z$ has some normalized marginal distribution:

$$Z \sim q(z)$$.

We wish to compute conditional expectations of the type: $E_{p(z \vert x, \theta)}[f(z)]$ but we don't know the this "posterior" for Z. E.g. We'd like to set $f=p(x,z|\theta)$ and find the likelihood of a given $\theta$, averaged over the possible values of Z given all the info we have available. But we can't, becuase we don't know $p(z \vert x, \theta)$.

Let's however say that we somehow know and can work with $q$. We can define the loss for using $q$ instead of $p(z \vert x, \theta)$ (abberviated as $p$) as the KL-divergence between $q$ and the posterior $p(z \vert x, \theta)$. This is a little backwards, given the interpretation of KL, but it's a valid loss function- it gets smaller the closer $p$ and $q$ are.

Then:

$$ D_{KL}(q, p) = E_q[log \frac{q}{p}] \\
= E_q\left[log \frac{q(z)}{p(z|x,\theta)}\right]\\
= -E_q\left[log \frac{p(z|x,\theta)}{q(z)}\right]$$

$$D_{KL}(q, p) = -E_q\left[log \frac{p(x, z \vert \theta)}{q(z)\,p(x \vert \theta)}\right]$$

We have used the definition of conditional probability in the last step: $p(z|x)=\frac{p(x,z)}{p(x)}$ whether or not we have a $\theta$ behind the bars everywhere.

So our goal would be to minimize the KL loss by choosing a good $\theta$.

Now at first blush this does not seem to have bought us anything as we don't know the full data likelihood $p(x, z \vert \theta)$ at all and we don't know how to optimize the x-data likelihood $p(x \vert \theta)$. But let's persevere:

$$D_{KL}(q, p) = - \left( E_q\left[log \frac{p(x, z \vert \theta)}{q(z)}\right] - E_q\left[log\,p(x \vert \theta)\right] \right)$$

The second term does not depend on $q$, and $q$ is normalized, so the expectation just gives back $log\,p(x \vert \theta)$.  Subtracting over, we can rewrite the equation in terns of the x-data log likelihoood thus:

$$log\,p(x \vert \theta) = E_q\left[log \frac{p(x, z \vert \theta)}{q(z)}\right] + D_{KL}(q, p)
$$

But wait! KL divergence is always 0 or larger. This means that the log likelihood is always greater than the ugly term. 

We name the ugly term the ELBO or Evidence Lower Bound. It depends on $q$ and $\theta$ (and technically on the observed data $x$, but those are fixed values):
$$log\,p(x \vert \theta) = ELBO(q,\theta) + D_{KL}(q, p) \\
log\,p(x \vert \theta) \ge ELBO(q,\theta)
$$

So we can find good $\theta$ and can maximize the likelihood $log\,p(x \vert \theta)$ by maximizing the ELBO.

To define and interpret the ELBO:
$$
ELBO = E_q\left[log \frac{p(x, z \vert \theta)}{q(z)}\right]\\
= E_q[log\, p(x, z \vert \theta)] - E_q[log\, q] \\
= E_q[-log\, q] - E_q[-log\, p(x, z \vert \theta)]
$$

The ELBO itself is the entropy of $q$, minus the average suprise at the complete data experienced by a particular $\theta$ (where the average is over the likely values of the missing data). For a given $q$ the entropy is fixed and the values of theta that are least surprised by the dataset have the biggest ELBO and thus the biggest upward pressure on $log\,p(x \vert \theta)$.

Unfortunately, the ELBO has slack: it depends on our approximating distribution $q$ as much as it does on $\theta$, and mucking with $\theta$ alone will give a good $\theta$ for that $q$ and that's about it. Moreover, likelihood is greater than ELBO, but there's no promise that better $\theta$ for a given $q$ actually increase the the likelihood: they may just increase ELBO and reduce the KL, leaving the likelihood unchanged.

To maximize ELBO in a meaningful way, we kind of need to find a good $q$, too, but selecting a good distribution out of a hat is hard and, it turns out, unecessary.

### A path
The relationship between ELBO, the overall likelihood, and the KL from the approximating distribution of Z and actual distribution of Z is a given theta is shown below (The oustide two always total to the middle).

$KL(q||p)$ is Bishop's notation for $D_{KL}(q, p)$. $\mathcal{L}(q, \theta)$ is the ELBO.

![](images/klsplitup.png)

### E-step
If we shrink the KL divergence, the ELBO is forced to grow to match and use up its slack; ELBO will be a tight bound on the likelihood.

![](images/klsplitestep.png)

At some (possibly initial) particular value of the parameters $\theta_{old}$, we can **fully** shrink the KL diveregence by choosing $q(z) = p(z  \vert  x, \theta_{old})$

This is the "good $q$" we were hoping for. It will set the Kullback Liebler divergence to 0, and thus set $\mathcal{L}(q, \theta)$ to the log-likelihood at $\theta_{old}$,  and maximizing the lower bound. We just approximate $p(z  \vert  x, \theta_{true})$ with $p(z  \vert  x, \theta_{old})$, but it's good enough to eliminate ELBO's slack.

Using this missing data posterior, conditioned on observed data, and $\theta_{old}$, we compute the expectation of the missing data with respect to the posterior and use it later.

### Now the **M-step**

After the E-step, the KL slack is gone and the ELBO touches the log-likelihood. Anything we do to $\theta$ to further increase the ELBO from its current value will also “push up” on the likelihood itself, at least a little.

Thus we hold now the distribution $q(z)$ fixed at the setting above, and maximize $\mathcal{L}(q, \theta)$ with respect to $\theta$ to obtain new parameter values $\theta_{new}$. 

The distribution $q=p(z  \vert  x, \theta_{old})$, we set above will not in general equal the new posterior distribution $p(z \vert x,\theta_{new})$, and hence there will be a nonzero KL divergence. Thus the increase in the log-likelihood we just created will be greater than the increase in the lower bound $\mathcal{L}$, as illustrated below.

The M in “M-step” and “EM” stands for “maximization”.

In [1]:
![](images/klsplitmstep.png)

'[]' is not recognized as an internal or external command,
operable program or batch file.


### The process

Note that since $\mathcal{L}(q,\theta)$ is maximized with respect to $\theta$, one can equivalently maximize the expectation of the full-data log likelihood $\mathrm{E_q[\ell( x,z  \vert  \theta)]}$ in the M-step since the difference is purely a function of $q$. Furthermore, if the joint distribution $p(x, z \vert  \theta)$ is a member of the exponential family, the log-likelihood will have a particularly simple form and will lead to a much simpler maximization than that of the incomple-data log-likelihood $p(x \vert \theta)$.

We now set $\theta_{old} = \theta_{new}$ and repeat the process. This **EM algorithm** is presented and  illustrated below:

![](images/emupdate.png)

1. We start with the log-likelihood $p(x  \vert  \theta)$(red curve) and the initial guess $\theta_{old}$ of the parameter values
2. Until convergence (the $\theta$ values dont change too much):
    1. E-step: Evaluate the hidden variable posterior $q(z, \theta_{old}) = p(z  \vert  x, \theta_{old})$ which gives rise to a lower bound function of $\theta$: $\mathcal{L}(q(z, \theta_{old}), \theta)$(blue curve) whose value equals the value of $p(x  \vert  \theta)$ at $\theta_{old}$.
    2. M-step: maximize the lower bound function with respect to $\theta$ to get $\theta_{new}$.
    3. Set $\theta_{old} = \theta_{new}$
    
One iteration more is illustrated above, where the subsequent E-step constructs a new lower-bound function that is tangential to the log-likelihood at $\theta_{new}$, and whose value at $\theta_{new}$ is higher than the lower bound at $\theta_{old}$ from the previous step.

Thus

$$\ell(\theta_{t+1}) \ge \mathcal{L}(q(z,\theta_t), \theta_{t+1}) \ge \mathcal{L}(q(z,\theta_t), \theta_{t}) = \ell(\theta_t)$$

The first equality follows since $\mathcal{L}$ is a lower bound on $\ell$, the second from the M-step's maximization of $\mathcal{L}$, and the last from the vanishing of the KL-divergence after the E-step. As a consequence, you **must** observe monotonic increase of the observed-data log likelihood $\ell$ across iterations. **This is a  powerful debugging tool for your code**.

**Warning: Local Maxes, Not Global**  
Note that as shown above, since each EM iteration can only improve the likelihood, you are guaranteeing convergence to a local maximum. Because it **IS** local , you must try some different initial values of $\theta_{old}$ and take the one that gives you the largest $\ell$.